In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob

In [62]:
def average_csv_files(file_pattern):
    csv_files = glob.glob(file_pattern)
    dfs = [pd.read_csv(file) for file in csv_files]
    
    combined_df = pd.concat(dfs)
    averaged_df = combined_df.groupby(['h_mm', 'h_MM', 'model', 'phase']).mean().reset_index()
    return averaged_df

file_pattern = "results/exploration_results_*.csv"
averaged_df = average_csv_files(file_pattern)

In [63]:
def plot_heatmaps(df, metric, save=False):
    before_df = df[df['phase'] == 'before']
    after_df = df[df['phase'] == 'after']
    data_before = before_df.pivot_table(index='h_MM', columns='h_mm', values=metric)
    data_after = after_df.pivot_table(index='h_MM', columns='h_mm', values=metric)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6)) 

    vmin = min(data_before.min().min(), data_after.min().min())
    vmax = max(data_before.max().max(), data_after.max().max())
    
    sns.heatmap(data_before, annot=False, cmap='coolwarm', fmt='.2f', cbar_kws={'label': metric}, vmin=vmin, vmax=vmax, ax=axes[0])
    axes[0].invert_yaxis()
    axes[0].set_title(f"Heatmap of {metric}-% (Before)")
    axes[0].set_xlabel('h_mm')
    axes[0].set_ylabel('h_MM')
    
    sns.heatmap(data_after, annot=False, cmap='coolwarm', fmt='.2f', cbar_kws={'label': metric}, vmin=vmin, vmax=vmax, ax=axes[1])
    axes[1].invert_yaxis()
    axes[1].set_title(f"Heatmap of {metric}-% (After)")
    axes[1].set_xlabel('h_mm')
    axes[1].set_ylabel('h_MM')

    plt.tight_layout()
    
    if save:
        plt.savefig(f"{metric}_before_after.png")
    else:
        plt.show()

In [ ]:
plot_heatmaps(averaged_df, metric='eo', save=False)  
plot_heatmaps(averaged_df, metric='sp', save=False)  

In [69]:
def plot_difference_heatmap(df, model=None):
    if model:
        df = df[df['model'] == model]
    before_df = df[df['phase'] == 'before']
    after_df = df[df['phase'] == 'after']

    before_sp = before_df.pivot_table(index='h_MM', columns='h_mm', values='sp')
    after_sp = after_df.pivot_table(index='h_MM', columns='h_mm', values='sp')
    diff_sp = after_sp - before_sp

    before_eo = before_df.pivot_table(index='h_MM', columns='h_mm', values='eo')
    after_eo = after_df.pivot_table(index='h_MM', columns='h_mm', values='eo')
    diff_eo = after_eo - before_eo

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    sns.heatmap(diff_sp, annot=False, cmap='coolwarm', fmt='.2f', cbar_kws={'label': 'Difference in sp (After - Before)'}, vmin=-10, vmax=10, ax=axes[0])
    axes[0].invert_yaxis()
    axes[0].set_title(f"sp-%")
    axes[0].set_xlabel('h_mm')
    axes[0].set_ylabel('h_MM')

    sns.heatmap(diff_eo, annot=False, cmap='coolwarm', fmt='.2f', cbar_kws={'label': 'Difference in eo (After - Before)'}, vmin=-10, vmax=10, ax=axes[1])
    axes[1].invert_yaxis()
    axes[1].set_title(f"eo-%")
    axes[1].set_xlabel('h_mm')
    axes[1].set_ylabel('h_MM')

    plt.show()

In [ ]:
plot_difference_heatmap(averaged_df)

In [198]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('results/exploration_results.csv')

df_diff = df.pivot_table(index=['h_mm', 'h_MM', 'fm', 'model'], columns='phase', values=['sp', 'eo'])
df_diff['sp_diff'] = df_diff[('sp', 'after')] - df_diff[('sp', 'before')]
df_diff['eo_diff'] = df_diff[('eo', 'after')] - df_diff[('eo', 'before')]
df_diff = df_diff.reset_index()

fig = make_subplots(
    rows=2, cols=3,
    shared_yaxes=True,
    shared_xaxes=True,
    column_titles=["fm=0.1", "fm=0.3", "fm=0.5"],
    row_titles=["Δsp (%)", "Δeo (%)"],
    x_title="h_mm",
    y_title="h_MM",
    vertical_spacing=0.02,
    horizontal_spacing=0.02
)

metrics = ['sp_diff', 'eo_diff']
fm_values = [0.1, 0.3, 0.5]
h_MM_values = sorted(df_diff['h_MM'].unique())

for row, metric in enumerate(metrics):
    for col, fm in enumerate(fm_values):
        df_filtered = df_diff[df_diff['fm'] == fm]
        heatmap_data = df_filtered.pivot(index='h_MM', columns='h_mm', values=metric)

        fig.add_trace(
            go.Heatmap(
                z=heatmap_data.values,
                x=heatmap_data.columns,
                y=heatmap_data.index,
                colorbar_title="After - Before",
                colorscale='RdBu',
                reversescale=True,
                zmid=0,
                zmin=-30,
                zmax=30
            ),
            row=row+1,
            col=col+1
        )

        fig.update_xaxes(tickmode='array', tickvals=h_MM_values)
        fig.update_yaxes(tickmode='array', tickvals=h_MM_values)

fig.update_layout(
    height=800,
    width=1200,
    showlegend=False
)
fig.show()

In [107]:
df_1 = pd.read_csv("graphtype_results_42_0.1.csv")
df_3 = pd.read_csv("graphtype_results_42_0.3.csv")
df_5= pd.read_csv("graphtype_results_42_0.5.csv")

In [196]:
def plot_bar(df):
    metrics = {
        'mean_acc': ('acc (%)', 'std_acc'),
        'mean_sp': ('Δsp (%)', 'std_sp'),
        'mean_eo': ('Δeo (%)', 'std_eo')
    }

    homophily_rates = {
        'Heterophilic': 0.22,
        'Hete/Homo': 0.36,
        'Neutral': 0.58,
        'Homo/Hete': 0.69,
        'Homophilic': 0.81
    }
    
    graph_types = df['graph'].unique()
    graph_labels = [f"{g}<br>({homophily_rates[g]:.2f})" for g in graph_types]
    fig = make_subplots(rows=1, cols=3, horizontal_spacing=0.04)

    for i, (metric, (ylabel, std_metric)) in enumerate(metrics.items()):
        clean_values = [df[(df['graph'] == g) & (df['phase'] == 'before')][metric].values[0] for g in graph_types]
        nifa_values = [df[(df['graph'] == g) & (df['phase'] == 'after')][metric].values[0] for g in graph_types]
        clean_std = [df[(df['graph'] == g) & (df['phase'] == 'before')][std_metric].values[0] for g in graph_types]
        nifa_std = [df[(df['graph'] == g) & (df['phase'] == 'after')][std_metric].values[0] for g in graph_types]

        fig.add_trace(go.Bar(
            x=graph_labels,
            y=clean_values,
            error_y=dict(type='data', array=clean_std, visible=True),
            name='Clean',
            marker=dict(color='#1f77b4'),
            offsetgroup='clean',
            hoverinfo="x+y",
            showlegend=(i == 0) 
        ), row=1, col=i+1)

        fig.add_trace(go.Bar(
            x=graph_labels,
            y=nifa_values,
            error_y=dict(type='data', array=nifa_std, visible=True),
            name='NIFA',
            marker=dict(color='#ff7f0e'),
            offsetgroup='nifa',
            hoverinfo="x+y",
            showlegend=(i == 0)  
        ), row=1, col=i+1)

        fig.update_yaxes(title_text=ylabel, row=1, col=i+1)

    fig.update_layout(
        legend=dict(
            orientation="h",
            xanchor="left",
            yanchor="bottom",
            x=0,
            y=1),
        barmode='group',
        width=1800,
        height=600,
        title_x=0.5
    )
    fig.update_xaxes(tickangle=0)

    fig.show()

In [197]:
plot_bar(df_3)